In [ ]:
import time
import numpy as np

from coppeliasim_zmqremoteapi_client import RemoteAPIClient 

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

In [ ]:
try:
    # Connect to the CoppeliaSim server
    client = RemoteAPIClient()
    sim = client.require("sim")
    sim.setStepping(True)
    
except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:

def quat_to_R(x, y, z, w):
    return np.array([
        [1-2*(y*y+z*z), 2*(x*y - z*w), 2*(x*z + y*w)],
        [2*(x*y + z*w), 1-2*(x*x+z*z), 2*(y*z - x*w)],
        [2*(x*z - y*w), 2*(y*z + x*w), 1-2*(x*x+y*y)]
    ], dtype=float)

def world_vertices_of_shape(h):
    viz = sim.getShapeViz(h, 0)             # pega o 1º submesh
    verts = np.array(viz['vertices'], float).reshape(-1, 3)  # coords no frame do shape
    pose = sim.getObjectPose(h, -1)         # [x,y,z,qx,qy,qz,qw]
    R = quat_to_R(*pose[4:]); t = np.array(pose[:3], float)
    verts_w = (verts @ R.T) + t
    return verts_w, np.array(viz['indices'], dtype=np.int32)

try:
    heightfield = sim.getObject('/heightfield')
    verticess_w, indices = world_vertices_of_shape(heightfield)

    # Vamos projetar X e Y, identificar a malha (grid) e montar Z[y, x]
    xy = verticess_w[:, :2]
    xvals = np.unique(np.round(xy[:,0], decimals=6))
    yvals = np.unique(np.round(xy[:,1], decimals=6))
    nx, ny = len(xvals), len(yvals)

    # Mapear cada (x,y) para índices i,j da grade
    # Observação: dependendo da triangulação, pode haver duplicatas; usamos argmin por proximidade
    def index_map(vals):
        return {v:i for i,v in enumerate(vals)}

    ix = index_map(xvals)
    iy = index_map(yvals)

    Z = np.full((ny, nx), np.nan, dtype=float)
    for x, y, z in verticess_w:
        j = ix[round(x,6)]
        i = iy[round(y,6)]
        # mantém o maior z (em geral já é único)
        if np.isnan(Z[i, j]) or z > Z[i, j]:
            Z[i, j] = z

    # Ordenação: y crescente corresponde às linhas; garanta que i=0 seja o menor y:
    y_order = np.argsort(yvals); x_order = np.argsort(xvals)
    Z = Z[y_order][:, x_order]
    xvals = xvals[x_order]; yvals = yvals[y_order]


    # ## Definir obstáculos por gradiente e altura
    # escolha seus limiares:
    z_thresh = np.nanmin(Z) + 0.05   # ex.: 5 cm acima do ponto mais baixo
    slope_thresh = 0.50              # ~ 26.6° (tan^-1(0.5)), ajuste conforme

    # calcula gradiente aproximado (metros de variação por metro no plano)
    dx = np.mean(np.diff(xvals)) if nx>1 else 1.0
    dy = np.mean(np.diff(yvals)) if ny>1 else 1.0

    dZdx = np.zeros_like(Z); dZdy = np.zeros_like(Z)
    dZdx[:,1:-1] = (Z[:,2:] - Z[:,:-2])/(2*dx)
    dZdy[1:-1,:] = (Z[2:,:] - Z[:-2,:])/(2*dy)
    slope = np.sqrt(dZdx**2 + dZdy**2)

    occ = (Z > z_thresh) | (slope > slope_thresh)   # matriz booleana de ocupação

    jj, ii = np.meshgrid(np.arange(nx), np.arange(ny))   # colunas, linhas
    centers = np.stack([xvals[jj], yvals[ii]], axis=-1).reshape(-1,2)  # XY de cada célula
    occupied_centers = centers[occ.reshape(-1)]          # lista de (x,y) de obstáculos

    # opcional: bounding boxes dos pixels (células)
    cell_dx, cell_dy = dx, dy
    # cada célula ocupa [x - dx/2, x + dx/2] × [y - dy/2, y + dy/2]

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
try:
    # Handle para o ROBÔ
    robotname = 'robotino'
    robotHandle = sim.getObject('/' + robotname)
    
    # Handle para as juntas das RODAS
    wheel1 = sim.getObject('/' + robotname + '/wheel0_joint')
    wheel2 = sim.getObject('/' + robotname + '/wheel1_joint')
    wheel3 = sim.getObject('/' + robotname + '/wheel2_joint')
    
    # Dados Robotino
    L = 0.135   # Metros
    r = 0.040   # Metros
               
    # Cinemática Direta
    Mdir = np.array([[-r/np.sqrt(3), 0, r/np.sqrt(3)], [r/3, (-2*r)/3, r/3], [r/(3*L), r/(3*L), r/(3*L)]])

    # Configuração inicial (x, y, w)
    q = np.array([0, 0, 0])

    # Goal configuration (x, y, theta)    
    qgoal = np.array([3, 3, np.deg2rad(90)])
    
    # Lembrar de habilitar o 'Real-time mode'    
    # Parar a simulação se estiver executando
    initial_sim_state = sim.getSimulationState()
    if initial_sim_state != 0:
        sim.stopSimulation()
        time.sleep(1)
    
    # Inicia a simulação
    sim.startSimulation()
    sim.step()


    # Frame que representa o Goal
    goalFrame = sim.getObject('/Goal')
    sim.setObjectPosition(goalFrame, [qgoal[0], qgoal[1], 0], sim.handle_world)
    sim.setObjectOrientation(goalFrame, [0, 0, qgoal[2]], sim.handle_world)
    gain = np.array([[0.1, 0, 0], [0, 0.1, 0], [0, 0, 0.1]])
    
    print("Starting robot control loop...")
    while (sim_time := sim.getSimulationTime()) <= 90:
        print(f"Simulation time: {sim_time:.2f} [s]")
        
        dt = sim.getSimulationTimeStep()
                
        pos = sim.getObjectPosition(robotHandle, sim.handle_world)
        ori = sim.getObjectOrientation(robotHandle, sim.handle_world)

        q = np.array([pos[0], pos[1], ori[2]])
        
        error = qgoal - q
        print(f"Error: {np.linalg.norm(error[:2])}")
        
        # Margem aceitável de distância
        if (np.linalg.norm(error[:2]) < 0.05):
            break

        # Controller
        qdot = gain @ error
        
        # Cinemática Inversa
        # w1, w2, w3
        Minv = np.linalg.inv(Rz(q[2]) @ Mdir)
        u = Minv @ qdot
        
        # Enviando velocidades
        sim.setJointTargetVelocity(wheel1, u[0])
        sim.setJointTargetVelocity(wheel2, u[1])
        sim.setJointTargetVelocity(wheel3, u[2]) 
                
        sim.step()

    # Parando o robô
    print("Stopping robot...")
    sim.setJointTargetVelocity(wheel1, 0)
    sim.setJointTargetVelocity(wheel2, 0)
    sim.setJointTargetVelocity(wheel3, 0)
    
    print('CALC Pos: ', sim_time, q[:2], np.rad2deg(q[2]))
      
    pos = sim.getObjectPosition(robotHandle, sim.handle_world)
    print('SIM Pos: ', pos)

    ori = sim.getObjectOrientation(robotHandle, sim.handle_world)
    print('SIM Ori: ', np.rad2deg(ori))
    

except Exception as e:
    print(f"An error occurred: {e}")

In [ ]:
# Parando a simulação
sim.stopSimulation()
print('Program ended')